In [2]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [3]:
#importing data 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [13]:
#Training Params 
learning_rate = 1e-3
training_steps = 1
batch_size = 128
display_step = 200

In [5]:
#Network parameters
num_input = 28 #image is 28*28
timesteps = 28
num_hidden = 128 #number of features that the hidden layer learns
num_classes = 10 #0-9 digits

In [6]:
#Inputs 
X = tf.placeholder(tf.float32, shape=[None, timesteps, num_input])
Y = tf.placeholder(tf.float32, shape=[None, num_classes])


In [7]:
#Weights 
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}

biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [15]:
def RNN(x, weights, biases):
    #Preparing data to match the rnn function requirements
    #Current data input shape: (batch_size, timesteps, num_input)
    #Required shape: "timesteps" tensors list of shape (batch_size, num_input)
    
    #Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1) 
    
    #LSTM cell 
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    #Get LSTM cell output
    #cell arg
    #inputs arg is a list of tensors. The shape of a tensor in the list is (batch_size, num_input)
    #and the list contains number of timesteps number of tensors; i.e., len(list) = timesteps
    
    outputs, states = rnn.static_rnn(cell=lstm_cell, inputs=x, dtype=tf.float32)
    
    
    #Linear activation, using rnn inner loop last output 
    #concerned about the ouput of the final time step, since the image would be "scanned" completely in 
    #the final timestep

    return tf.matmul(outputs[-1], weights['out']) + biases['out']


In [9]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

#loss and optimizer 
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#Evaluating the model 
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#Global variables initializer
init = tf.global_variables_initializer()


In [17]:
#Training 
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
    print("Optimization Finished!")
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 2.8292, Training Accuracy= 0.070
Optimization Finished!
Testing Accuracy: 0.09375
